In [ ]:
import requests
from bs4 import BeautifulSoup

In [48]:
import requests
from bs4 import BeautifulSoup

def get_city_info_wikipedia(city_name):
    city_information=""
    # Passo 1: Primeiro, fazemos uma consulta para obter o ID da página da cidade
    url = f'https://pt.wikipedia.org/w/api.php?action=query&format=json&titles={city_name}'

    response = requests.get(url)
    data = response.json()

    # Extraindo o ID da página da resposta
    pages = data['query']['pages']
    page_id = next(iter(pages))  # Pegando o primeiro ID da página (único)
    
    # Passo 2: Consultar o conteúdo da página usando o ID obtido
    if page_id != '-1':  # Se a página existir
        city_information += f"Informações sobre a cidade: {city_name}\n"
        page_url = f'https://pt.wikipedia.org/wiki/{city_name.replace(" ", "_")}'
        city_information += f"Link para mais detalhes: {page_url}\n\n"
        
        # Requisição para pegar o conteúdo da página
        url_details = f'https://pt.wikipedia.org/w/api.php?action=parse&format=json&pageid={page_id}&prop=text&formatversion=2'
        response_details = requests.get(url_details)
        data_details = response_details.json()

        # Pegando o conteúdo (HTML da página)
        content = data_details['parse']['text']
        
        # Usando BeautifulSoup para parsear o HTML
        soup = BeautifulSoup(content, 'html.parser')

        # Exemplo de busca por informações específicas de Clima, Economia, Educação, Transporte, População
        # Buscando essas informações com base nas seções encontradas

        city_information += f"Informações encontradas sobre:{city_name}\n"
        
        # Buscando seções de interesse
        for section in soup.find_all(['h2', 'h3', 'h4']):
            section_text = section.get_text(strip=True).lower()
    
            if any(keyword in section_text for keyword in ['clima', 'economia', 'educação', 'transporte', 'população','lazer','bairros','gastronomia',
                                                           'parques e jardins','demografia','turismo','saude','desporto']):
                city_information += f"\n{section_text.capitalize()}:\n"
                
                # Pegando o conteúdo da seção e exibindo
                section_next = section.find_next(['p', 'ul'])
                if section_next:
                    city_information += f"- {section_next.get_text(strip=True)}\n"
        
 # Exemplo: Extrair tabelas (como infoboxes)
        city_information += "\nInfobox encontrados:"
        infobox = soup.find('table', {'class': 'infobox'})
        if infobox:
            # Para cada linha da tabela do infobox
            for row in infobox.find_all('tr'):
                # Encontrando os <td> de cada linha (dados e valores)
                cells = row.find_all('td')
                if len(cells) == 2:  # A linha tem 2 células (título e valor)
                    title = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)

                    # Exibe o título e o valor extraído
                    city_information += f"- {title}: {value}\n"
                    
                    # Caso haja links, podemos mostrar o URL
                    
                    # link = cells[1].find('a')
                    # if link:
                    #     link_url = link.get('href')
                    #     print(f"  Link: {link_url}")

    else:
        city_information += f"Página '{city_name}' não encontrada."

    return city_information

In [53]:
# Exemplo de uso:
city_name = 'Setúbal'
city_info = get_city_info_wikipedia(city_name)


In [54]:
filepath = '..\\..\\Bases\\Wikipedia\\' + city_name + '.txt'
# save information to a file
with open(filepath, 'w', encoding='utf-8') as file:
    file.write(city_info)   